# File I/O

MolPy’s IO layer is built around `Frame` + `Box`.  
Readers return a `Frame` (with blocks like `"atoms"`, `"bonds"` and a box in `metadata`),
and writers take a `Frame` and produce on‑disk files compatible with external tools.

This page gives you **small, runnable examples** for the most common formats.

---

## PDB: quick coordinate round‑trip

PDB is convenient for small molecules and quick visualization.



In [ ]:
import molpy as mp
from molpy.io import read_pdb, write_pdb

# Build a tiny system in a frame
box = mp.Box.cubic(20.0)

frame = mp.Frame()
frame["atoms", "x"] = [0.0, 1.0, 2.0]
frame["atoms", "y"] = [0.0, 0.0, 0.0]
frame["atoms", "z"] = [0.0, 0.0, 0.0]
frame["atoms", "element"] = ["C", "C", "C"]
frame.metadata["box"] = box

write_pdb("example.pdb", frame)
loaded = read_pdb("example.pdb")

print("Loaded atoms:", loaded["atoms"].nrows)


Under the hood:

- `read_pdb` populates:
  - `frame["atoms"]` with an atoms `Block` (including `xyz` coordinates)
  - `frame["bonds"]` from `CONECT` records when available
  - `frame.metadata["box"]` from `CRYST1` where possible
- `write_pdb` writes:
  - `ATOM`/`HETATM` records from `frame["atoms"]`
  - `CRYST1` from the box
  - `CONECT` from the `"bonds"` block

---

## LAMMPS data: structure for simulations

MolPy can read and write LAMMPS “data” files using `Frame` as the in‑memory representation.

### Writing a LAMMPS data file



In [ ]:
from molpy.io import write_lammps_data

# Assume `frame` has been constructed or loaded before
write_lammps_data("system.data", frame, atom_style="full")


The writer expects at least:

- `"atoms"` block with the required columns for the chosen `atom_style`
- `"bonds"` / `"angles"` / `"dihedrals"` blocks when present
- A `Box` stored in `frame.metadata["box"]`

### Reading a LAMMPS data file



In [ ]:
from molpy.io import read_lammps_data

frame = read_lammps_data("system.data", atomstyle="full")
print("Atoms:", frame["atoms"].nrows)
print("Has bonds:", "bonds" in list(frame.blocks()))


For legacy workflows there is also `molpy.io.read_lammps(...)` which can optionally
load force‑field scripts; for new code, prefer `read_lammps_data`.

---

## LAMMPS trajectories

Trajectory readers return a `Frame` per snapshot.  
MolPy’s LAMMPS trajectory reader is designed to be lazy and memory‑friendly.



In [ ]:
from molpy.io.trajectory.lammps import LammpsTrajectoryReader

reader = LammpsTrajectoryReader("traj.lammpstrj")

for i, frame in enumerate(reader):
    print("Step", i, "atoms:", frame["atoms"].nrows)
    if i >= 9:
        break  # just peek at the first 10 frames


Use this pattern when you want to stream through long trajectories without
loading everything into memory at once.

---

## AMBER & GROMACS systems

MolPy also provides convenience functions for reading systems from AMBER and GROMACS.

### AMBER



In [ ]:
from molpy.io import read_amber

frame, ff = read_amber("system.prmtop", "system.inpcrd")
print("Atoms:", frame["atoms"].nrows)
print("Box:", frame.metadata.get("box"))


### GROMACS



In [ ]:
from molpy.io import read_gromacs

frame, ff = read_gromacs("conf.gro", "topol.top")
print("Atoms:", frame["atoms"].nrows)


These helpers return:

- A `Frame` describing the atomic system (+ `Box` in metadata)
- A force‑field object you can later map into MolPy’s force‑field layer

---

## Other formats

MolPy’s IO layer is designed to be extensible. In addition to the formats above,
there are readers/writers for:

- XSF (`read_xsf`, `write_xsf`)
- XYZ and simple CSV‑like formats (via `Block.from_csv` and small utilities)
- Additional trajectory readers under `molpy.io.trajectory`

Most of them follow the same pattern:

- **Input**: file path + optional existing `Frame`
- **Output**: populated `Frame` with `"atoms"` (and optionally `"bonds"`, `"box"`, …)

When in doubt, look at the corresponding module under `molpy.io.*` and the
unit tests under `tests/io` for the exact expectations.

